In [1]:
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
import numpy as np
from collections import Counter
import seaborn as sns



Duplicate key in file WindowsPath('c:/Users/USER/anaconda3/envs/python3.9.15/lib/site-packages/matplotlib/mpl-data/matplotlibrc'), line 273 ('font.monospace:  DejaVu Sans Mono, Bitstream Vera Sans Mono, Computer Modern Typewriter, Andale Mono, Nimbus Mono L, Courier New, Courier, Fixed, Terminal, monospace')


In [2]:
course_chapter_items_df = pd.read_csv('./data/course_chapter_items.csv')
course_df = pd.read_csv('./data/courses.csv')
users_df = pd.read_csv('./data/users.csv')
subgroups_df = pd.read_csv('./data/subgroups.csv')

train_group_df = pd.read_csv('./data/train_group.csv')
test_seen_group_df = pd.read_csv('./data/test_seen_group.csv')
val_seen_group_df = pd.read_csv('./data/val_seen_group.csv')
test_unseen_group_df = pd.read_csv('./data/test_unseen_group.csv')
val_unseen_group_df = pd.read_csv('./data/val_unseen_group.csv')

train_df = pd.read_csv('./data/train.csv')
test_seen_df = pd.read_csv('./data/test_seen.csv')
val_seen_df = pd.read_csv('./data/val_seen.csv')
test_unseen_df = pd.read_csv('./data/test_unseen.csv')
val_unseen_df = pd.read_csv('./data/val_unseen.csv')

In [54]:
course_df.head(2)

,course_id,course_name,course_price,teacher_id,teacher_intro,groups,sub_groups,topics,course_published_at_local,description,will_learn,required_tools,recommended_background,target_group
0,61888e868f154b000781b45a,少女人妻華麗變身：七大妝容七彩的夢幻樂園,1800,61888e7bb7fe1c0006850eff,在美妝 KOL 圈裡屬個人風格強烈的 Alice，在清新與叛逆風格間遊刃有餘，其幽默的美妝影...,生活品味,"更多生活品味,護膚保養與化妝","更多生活品味,護膚保養與化妝",NaN,"<img src=""https://images.api.hahow.in/images/6...",不再害怕各種顏色的彩妝，可以更隨心搭配各種繽紛的顏色。\n,所需工具為：視課程實際會用到的彩妝用品,只要你有一顆愛化妝、想變漂亮的心皆可以參加。\n\n⚠️ 雖然課程當中會帶到相關彩妝技巧，不...,熱愛彩妝的人
1,54d5a117065a7e0e00725ac0,幾何圖形分割 X 色塊組合,100,54d5a079065a7e0e00725abe,從學生時代開始摸索photoshop等軟體，自以為有些天賦但後來發現其實沒有。出社會後從事美...,"藝術,設計","平面設計,繪畫與插畫","Illustrator/以拉,配色技巧",2015-03-27 23:38:27.195,"<img src=""https://storage.googleapis.com/hahow...",可以將任何區塊分割成自己想要的幾何圖形、快速的上色，並且能夠應用在許多地方,"Adobe Illustrator（必備）, camera",知道如何使用Illustrator的基本工具列,每一位興趣的人都能學得來，非常容易的小技巧


In [32]:
id2course_mapping = course_df["course_id"].to_dict()
course2id_mapping = {v : k for k, v in id2course_mapping.items()}
len(course2id_mapping)

728

In [75]:
def clean_data(x):
        return x.replace(" ", "")

filledna=course_df.fillna('')
features= ['course_name', 'teacher_intro', 'sub_groups', 'topics', 'will_learn', 'target_group']
filledna=filledna[features]
for feature in features:
    filledna[feature] = filledna[feature].apply(clean_data)
    
filledna.head(2)

,course_name,teacher_intro,sub_groups,topics,will_learn,target_group
0,少女人妻華麗變身：七大妝容七彩的夢幻樂園,在美妝KOL圈裡屬個人風格強烈的Alice，在清新與叛逆風格間遊刃有餘，其幽默的美妝影片手法...,"更多生活品味,護膚保養與化妝","更多生活品味,護膚保養與化妝",不再害怕各種顏色的彩妝，可以更隨心搭配各種繽紛的顏色。\n,熱愛彩妝的人
1,幾何圖形分割X色塊組合,從學生時代開始摸索photoshop等軟體，自以為有些天賦但後來發現其實沒有。出社會後從事美...,"平面設計,繪畫與插畫","Illustrator/以拉,配色技巧",可以將任何區塊分割成自己想要的幾何圖形、快速的上色，並且能夠應用在許多地方,每一位興趣的人都能學得來，非常容易的小技巧


In [76]:
def create_soup(x):
    return x['course_name']+ ' ' + x['teacher_intro'] + ' ' + x['sub_groups'] + ' ' + x['topics'] + ' ' + x['will_learn'] + ' ' + x['target_group'] 

filledna['soup'] = filledna.apply(create_soup, axis=1)

In [77]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel


tfidf = TfidfVectorizer(stop_words='english')
#Construct the required TF-IDF matrix by fitting and transforming the data
tfidf_matrix = tfidf.fit_transform(filledna['soup'])
# Compute the cosine similarity matrix
cosine_sim2 = linear_kernel(tfidf_matrix, tfidf_matrix)
#Output the shape of tfidf_matrix
tfidf_matrix.shape


# count = CountVectorizer(stop_words='english')
# count_matrix = count.fit_transform(filledna['soup'])

# cosine_sim2 = cosine_similarity(count_matrix, count_matrix)

(728, 20998)

In [83]:
def get_recommendations_new(haved_courses_list, cosine_sim, top = 10):
    cosine_sim_sum = [ [i, 0] for i in range(len(course2id_mapping))]
    haved_courses_index_list = [ course2id_mapping[course_id] for course_id in haved_courses_list]
    for idx in haved_courses_index_list:
        # Get the pairwsie similarity scores of all courses with that course
        sim_scores = list(enumerate(cosine_sim[idx]))
        for i in range(len(sim_scores)):
            cosine_sim_sum[i][1] += sim_scores[i][1]

    # Sort the movies based on the similarity scores
    sim_scores = sorted(cosine_sim_sum, key=lambda x: x[1], reverse=True)

    recommend_indices = []
    # Get the scores of the 10 most course not buy
    for i in range(len(sim_scores)):
        if len(recommend_indices) < top:
            if sim_scores[i][0] not in haved_courses_index_list:
                recommend_indices.append(sim_scores[i][0])

    # Return the top 10 most similar movies
    return course_df['course_id'].iloc[recommend_indices].tolist()
    
get_recommendations_new(['5513e92b38239d10005778e1', '54f1268f4ec3c809002e4a29'], cosine_sim2, 20)

['插一盆花歌頌四季－我的自然感花藝創作法',
 '手繪珠寶設計入門：完成你的第一件珠寶設計',
 'NFT 加密藝術｜買幣、發行、交易一次上手！',
 '靈感不枯竭！30 種方式提升內容創作力',
 '攝影創作 - 調出電影感色彩',
 'Seagate 講堂 | 影像後製再現影像教你做履歷',
 'Seagate 講堂 | 動態導演林呈軒教你做履歷',
 '影片製作入門10堂課 - 攝影、剪輯、腳本',
 '超現實影像創作 - PS 合成後製課',
 '前進夢想工作：寫出必勝履歷！',
 '職場英文大補帖｜銀行行員必備的英文口說',
 '職場英文大補帖｜必勝簡報方程式',
 'Seagate 講堂 | 平面設計師顏伯駿教你做履歷',
 '一個小時！DIY你的作品集網站',
 '日本求職全攻略，開啟你海外求職第一步',
 '作品集必備！Mockup 模擬圖合成術',
 '履歷面試大剖析－網路科技外商求職技巧',
 '職場英文大補帖｜從社交心法到溝通金句',
 '新多益閱讀：英語教父助你一舉突破企業門檻',
 '雙語教育必學的核心素養－用英文學知識']

In [81]:
seen_predict_df = pd.read_csv('./data/test_seen.csv')
predict_user = seen_predict_df["user_id"].to_list()


seen_user_haved_purchased_course = {}
for seen_user_id, course_ids in zip(train_df["user_id"], train_df["course_id"]):
    course_id_list = course_ids.split(' ')
    seen_user_haved_purchased_course[seen_user_id] = seen_user_haved_purchased_course.setdefault(seen_user_id, []) + course_id_list
for seen_user_id, course_ids in zip(val_seen_df["user_id"], val_seen_df["course_id"]):
    course_id_list = course_ids.split(' ')
    seen_user_haved_purchased_course[seen_user_id] = seen_user_haved_purchased_course.setdefault(seen_user_id, []) + course_id_list
print(len(seen_user_haved_purchased_course))

59737


In [84]:
import csv

predict_users = seen_predict_df["user_id"].to_list()
with open("predict.csv", "w") as f:
    writer = csv.writer(f)
    writer.writerow(["user_id", "course_id"])
    for user_id in predict_users:
        recommend = " ".join(get_recommendations_new(seen_user_haved_purchased_course[user_id], cosine_sim2, top = 50))
        writer.writerow([user_id, recommend])